# 概要
ANNs(Artificial neural networks)人工神经网络是深度学习的核心技术，适用于高度复杂的机器学习任务，比如分类百万级的图片(Google Image),语音识别(Apple's Siri),或者是打败世界冠军(Alpha Go).
# 感知机
只有一层输入层(包含Bias Neuron和input Neuraon)和一层输出层(LTU).其中LTU：  
$h_{\mathbf w}(x)=step(z)=step(\mathbf w^T\cdot\mathbf x)$  
Heaviside step function  或者是符号函数  
$haviside(z)=\left{
\begin{array}{ll}
0&\text {if }z<0\\
1&\text {if }z\ge0\\ 
\end{array}
\right.$  

$sgn(z)=\left{
\begin{array}{ll}
1&\text {if }z<0\\
0&\text {if }z=0\\ 
-1&\text {if }z<0
\end{array}
\right. $  
感知机的训练规则：误差逆传播，每次只输入一个样例，分别对他们做出预测，如果出现错误预测，就加强导致输出错误的输入神经元连接权值。    

$w_{i,j}^{(next step)}=w_{i.j}+\eta(\hat y_j-y_j)x_i$  
$w_{i,j}是第i个输入神经元和第j个输出神经元间的连接权值$  
$x_i是当前样例的第i个输入神经元的值$  
$\hat y_j是当前样例第j个输出神经元的输出值$  
$y_j是当前样例的第j个输出神经元的目标值$  
$\eta 是学习率$  
感知机的每个输出神经元的决策限是线性的，所以不能学习复杂的模式。

In [1]:
#sklearn提供Perception类,是一个单一的LTU网络  
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:,(2, 3)]
y = (iris.target == 0).astype(np.int)
per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

y_pred = per_clf.predict([[2, 0.5]])
y_pred

G:\machine-learn\anaconda\envs\tensorflow\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


array([1])

感知机实际就是SGD的集成模式，实际上它等价于SGD,其中loss='perception'，learning_rate='constant'.eta0=1,penalty=None的情况.  
注意到与逻辑回归分类相反的是，感知机不输出每个类的概率，它只基于阈值给出预测结果。所以逻辑回归比感知机优秀。  
单层的神经网络分类效果很差，甚至连XOR问题都解决不了。  
# 多层感知和逆向传播
多层感知机包含有一个或多个由LTU构成的中间层(隐层)，除了输出层外，每层都有偏置神经元，并且每层之间全连接。当ANN含有两个或更多隐层时，称作DNN（deep neural networl深度神经网络).  
多年以来，研究者都无法找到训练MLPs的方法。知道逆向传播算法的出现。如今我们使用reverse-mode autodiff（逆向自动微分）的梯度下降来描述这种算法。  
  
对于每一个训练样例，这个算法反馈到网络并计算每个层的输出。然后测算每个层的输出误差。然后计算上一层的每个神经元对该层输出误差的贡献，逐个往前直到输入层。使用这种逆向的方式对每个连接权值求误差对其的梯度。这种方式对逆向自动微分来说十分简单。最后的调整权值来减小误差。
  
为了让算法可以工作，需要在MLPs结构上做关键改动：将step函数改为逻辑函数$\sigma(z)=1/(1+e^{-z})$,这就是的梯度可求，逆向传播算法还有其他的激活函数，可用来替代，比如:  
hyperbolic tangent function  $tanh(z)=2\sigma(2z)-1$  
一样是s型可微函数，但是输出是-1到1,使得每个蹭的输出对于训练开始时越来越规范化，这有助于加速收敛。  
ReLU function $ReLU(z)=max(0,z)$  
连续不可微。然而在实际应用中表现很好，可以加快运算，最重要的是，输出没有上线，可以减少一些梯度下降的问题。  
  
一个MLP通常用来分类，因为每个输出是二分类的,当面对多分类的问题时，可以在最后层输出加入softmax函数，每个输出神经元对应输出类的概率。单向船体的神经网络结构称为前馈神经网络(feedforward neural netweorFNN)  
# 使用TF的高级API训练MLP
一个训练MLP的最简单方法就是使用TF的高级API-TF.Learn。和sklearn很相似。DNNClassifier创建一个DNN神经网络包含若干隐层，输出可以对类别做概率求值。

In [41]:
#为了使用tb
#from datetime import datetime

#now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_Logs'
logdir = "{}/2_Introduce for ANN/".format(root_logdir)


In [21]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

#load data
X = load_digits().data.astype(np.int)
y = load_digits().target.astype(np.int)

#scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42, test_size=0.8, shuffle=True)

#model construct
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=10,  #两个隐层一个300个神经元，另一个100个
                                        feature_columns=feature_columns)
#model train
dnn_clf.fit(x=X_train, y=y_train, batch_size=50, steps=40000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001DA8896DFD0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\CAIJIN~1\\AppData\\Local\\Temp\\tmpqsdocf51'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\CAIJIN~1\AppData\Local\Temp\tmpqsdocf51\model.ckpt.
INFO:tenso

INFO:tensorflow:loss = 7.460798e-05, step = 6801 (0.273 sec)
INFO:tensorflow:global_step/sec: 374.265
INFO:tensorflow:loss = 0.00019611201, step = 6901 (0.267 sec)
INFO:tensorflow:global_step/sec: 363.381
INFO:tensorflow:loss = 0.000102598606, step = 7001 (0.275 sec)
INFO:tensorflow:global_step/sec: 260.912
INFO:tensorflow:loss = 0.000111172885, step = 7101 (0.383 sec)
INFO:tensorflow:global_step/sec: 384.344
INFO:tensorflow:loss = 0.00011363651, step = 7201 (0.259 sec)
INFO:tensorflow:global_step/sec: 378.52
INFO:tensorflow:loss = 0.00011220418, step = 7301 (0.265 sec)
INFO:tensorflow:global_step/sec: 350.629
INFO:tensorflow:loss = 0.00016721395, step = 7401 (0.284 sec)
INFO:tensorflow:global_step/sec: 344.583
INFO:tensorflow:loss = 0.00011923719, step = 7501 (0.291 sec)
INFO:tensorflow:global_step/sec: 287.152
INFO:tensorflow:loss = 0.00011837533, step = 7601 (0.349 sec)
INFO:tensorflow:global_step/sec: 355.622
INFO:tensorflow:loss = 0.00010915144, step = 7701 (0.280 sec)
INFO:tensor

limit_output extension: Maximum message size of 10000 exceeded with 10019 characters

In [22]:
from sklearn.metrics import accuracy_score
y_pred = list(dnn_clf.predict(X_test))
accuracy_score(y_test, y_pred)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\CAIJIN~1\AppData\Local\Temp\tmpqsdocf51\model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.9415855354659249

In [23]:
#或者使用tf自己的评估函数
dnn_clf.evaluate(X_test, y_test)

INFO:tensorflow:Starting evaluation at 2018-03-19-07:36:21
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\CAIJIN~1\AppData\Local\Temp\tmpqsdocf51\model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-19-07:36:21
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.94158554, global_step = 40000, loss = 0.29512927


{'accuracy': 0.94158554, 'global_step': 40000, 'loss': 0.29512927}

# 使用tf清晰地训练DNN
如果你想要完全控制网络结构，就要使用TF的低层次API。  
## 构造层
首先我们需要导入tf库，然后特例化输出和输入数目，每个隐层的神经元数目。

In [32]:
import tensorflow as tf

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

#然后使用placeholder来表示X，y
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.float32, shape=(None), name='y')

#接着构造网络
## 首先构造神经层
def neuron_layer(X, n_neurons, name, activation=None):   
    with tf.name_scope(name):                               #定义了名字域，将包含整个神经层
        n_inputs = int(X.get_shape()[1])                       #查看数组型，并返回第二维的数量，第一维是输入的样例数目
        stddev = 2 / np.sqrt(n_inputs)                        #紧接的三行创建了W-权值矩阵，它的型是(n_inputs,n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)  #使用truncated标准高斯分布来随机化权值，使用标准差
        W = tf.Variable(init, name='weights')                    #为2/np.sqrt(n_inputs),这个值有助于算法快速收敛
        b = tf.Variable(tf.zeros([n_neurons]), name='bias')          #初始化每个神经元的偏置项
        z = tf.matmul(X, W) + b                             #创建子图计算矩阵乘和加法，
        if activation=='relu':                              #激活函数，是否是relu
            return tf.nn.relu(z)
        else:
            return z
        
#下面就是使用构造函数来建立DNN！
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(X, n_hidden1, 'hidden1', activation='relu')
    hidden2 = neuron_layer(hidden1, n_hidden2, 'hidden2', activation='relu')
    logits = neuron_layer(hidden2, n_outputs, 'outputs')              #这里并不对输出做softmax，方便管理层结构


#然后将下列函数加入构造层末尾  
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
file_writer.close()

In [106]:
#其实TF有用来构造标准神经网络的函数，比如fully_connected()可以建立全连接层，这
#个函数更加关注权值和偏置变量，使用合适的初始化，并且默认使用relu，并且提供一
#些标准化和正则化的参数,下面是使用他来替换前面的构造网络，两者等价
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

from tensorflow.contrib.layers import fully_connected

with tf.name_scope('dnn'):
    hidden1 = fully_connected(X, n_hidden1, scope='hidden1')
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2')
    logits = fully_connected(hidden2, n_outputs, scope='outputs',
                            activation_fn=None)

#tf.contrib提供很多有用的函数，不过这个包是实验代码的包，不是主要的API，所以可
#能会被移除

现在我们有了可供运行的网络结构，我们还需要定义一个损失函数才能训练它，我们使用交叉熵来作为损失函数，这样模型的输出的概率数值会尽量低，TF提供计算交叉熵的函数。这个将给对每个样例返回一个1d的tensor，我们接着使用TF的reduce_mean()来计算所有样例的平均交叉熵：

In [107]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=y, logits=logits)                        #默认将标签onehot，输出softmax
    loss = tf.reduce_mean(xentropy, name='loss')

这里我们已经有了网络，有了损失函数，现在我们需要添加梯度下降优化器，来调整模型参数，以最小化损失函数，

In [108]:
learing_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learing_rate)
    training_op = optimizer.minimize(loss)

还有一个重要的步骤就是特例化评价模型的函数。我们简单的使用准确度。首先，对于每个样例，判断预测概率最高的项是否符合真正的类，这个可以使用in_top_k()函数，返回一个1Dtensor包含了boolean值，我们要把bool值转换成float然后计算平均值，

In [109]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

然后我们创建一个节点来初始化所有变量，并且创建一个Saver来把模型保存到硬盘

In [110]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

acc_summary_train = tf.summary.scalar('acc_train', accuracy)
acc_summary_test = tf.summary.scalar('acc_test', accuracy)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

# 执行层
这部分更加短和简单，首先加载MNIST，使用TF自己的MNIST数据，它会自动的抓取，变换，打乱这些数据，还简单的提供一个函数每次只加载mini-batch

In [111]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [112]:

#然后定义执行次数和mini-batch的大小
n_epochs = 400
batch_size = 50

#训练模型
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                y: mnist.test.labels})
        
        
        acc_summary_train_ = acc_summary_train.eval(feed_dict={X: X_batch, y: y_batch})  
        file_writer.add_summary(acc_summary_train_, epoch)
        
        acc_summary_test_ = acc_summary_test.eval(feed_dict={X: mnist.test.images,
                                           y: mnist.test.labels})   
        file_writer.add_summary(acc_summary_test_, epoch)
        
        print(epoch, 'Train accuracy:', acc_train, 'Test accuracy:', acc_test)
    
    save_path = saver.save(sess, './my_model_final.ckpt')
    
file_writer.close()

0 Train accuracy: 0.9 Test accuracy: 0.9024
1 Train accuracy: 0.94 Test accuracy: 0.92030007
2 Train accuracy: 0.97999996 Test accuracy: 0.92910004
3 Train accuracy: 0.96 Test accuracy: 0.93520004
4 Train accuracy: 0.91999996 Test accuracy: 0.94100004
5 Train accuracy: 0.96 Test accuracy: 0.94670004
6 Train accuracy: 0.96 Test accuracy: 0.94890004
7 Train accuracy: 0.91999996 Test accuracy: 0.9538
8 Train accuracy: 0.97999996 Test accuracy: 0.9556
9 Train accuracy: 0.96 Test accuracy: 0.9584001
10 Train accuracy: 0.97999996 Test accuracy: 0.9592
11 Train accuracy: 0.97999996 Test accuracy: 0.96300006
12 Train accuracy: 0.97999996 Test accuracy: 0.96230006
13 Train accuracy: 1.0 Test accuracy: 0.96520007
14 Train accuracy: 0.97999996 Test accuracy: 0.9678
15 Train accuracy: 1.0 Test accuracy: 0.96830004
16 Train accuracy: 1.0 Test accuracy: 0.9696
17 Train accuracy: 1.0 Test accuracy: 0.9707
18 Train accuracy: 0.96 Test accuracy: 0.97090006
19 Train accuracy: 0.97999996 Test accuracy: 0

167 Train accuracy: 1.0 Test accuracy: 0.98020005
168 Train accuracy: 1.0 Test accuracy: 0.9804
169 Train accuracy: 1.0 Test accuracy: 0.98050004
170 Train accuracy: 1.0 Test accuracy: 0.98020005
171 Train accuracy: 1.0 Test accuracy: 0.9804
172 Train accuracy: 1.0 Test accuracy: 0.98
173 Train accuracy: 1.0 Test accuracy: 0.98020005
174 Train accuracy: 1.0 Test accuracy: 0.98020005
175 Train accuracy: 1.0 Test accuracy: 0.98020005
176 Train accuracy: 1.0 Test accuracy: 0.9804
177 Train accuracy: 1.0 Test accuracy: 0.98
178 Train accuracy: 1.0 Test accuracy: 0.98010004
179 Train accuracy: 1.0 Test accuracy: 0.98020005
180 Train accuracy: 1.0 Test accuracy: 0.98020005
181 Train accuracy: 1.0 Test accuracy: 0.98020005
182 Train accuracy: 1.0 Test accuracy: 0.97990006
183 Train accuracy: 1.0 Test accuracy: 0.98010004
184 Train accuracy: 1.0 Test accuracy: 0.98010004
185 Train accuracy: 1.0 Test accuracy: 0.9804
186 Train accuracy: 1.0 Test accuracy: 0.97980005
187 Train accuracy: 1.0 Test

limit_output extension: Maximum message size of 10000 exceeded with 10042 characters

# 使用已经训练好的神经网络
现在已经训练好模型，可以使用它来做预测，需要重用前面的图，然后修改执行层如下，把已经训练好的参数加载下来。

In [116]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

from tensorflow.contrib.layers import fully_connected

with tf.name_scope('dnn'):
    hidden1 = fully_connected(X, n_hidden1, scope='hidden1')
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2')
    logits = fully_connected(hidden2, n_outputs, scope='outputs',
                            activation_fn=None)

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=y, logits=logits)                        #默认将标签onehot，输出softmax
    loss = tf.reduce_mean(xentropy, name='loss')

learing_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learing_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

acc_summary_train = tf.summary.scalar('acc_train', accuracy)
acc_summary_test = tf.summary.scalar('acc_test', accuracy)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

#修改执行层

with tf.Session() as sess:
    saver.restore(sess, './my_model_final.ckpt')
    X_new_scaled = mnist.test.images
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)
file_writer.close()

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


# 调优神经网络的参数
神经网络的灵活性同样是它的缺点，有太多参数需要调整，不仅是网络的拓扑结构，甚至MLP都可以改变不同层数，不同数量神经元，激活函数类型，权重初始化的逻辑，如何找到最优值呢：  
当然可以用gridsearch方式，但是当网络比较大，数据比较多的时候，就没有这么多时间去尝试了。所以最好用randomsearch；另外一个可选项是，使用Oscar这样的工具。  
## 隐层数目
对于大多数问题，可以从单隐层开始，然后找到一个合理的结果，实际上只含有一个隐层的MLP都能模拟复杂的函数，只要有足够的神经元，长久以来，这对于研究者们很方便，不需要去使用DNN，但是随后他们发现DNN对于参数利用率极高，他们只需要一些神经元就能模拟复杂函数，而且训练也很快。  
分层结构不仅仅使DNN快速，还使它能适应新的数据，新的模型甚至可以使用旧模型的参数来初始化其低层。  
总的来说，对于多数问题，只需要1到2层隐层。对于一些更复杂的问题，可以逐渐增加，同时也需要大量的训练样例。然而，你几乎不需要从头训练这样的模型，他们有很多相似任务，是已经训练好的网络的重复，这使得训练很快而且不需要那么多数据。
## 每个隐层神经元数目
显然输入层和输出层的神经元数目对应任务所需要的输入输出数目。对于隐层，一个通常的手段是做成漏斗形，逐层递减。但是这个手段现在不是很通用了，你也可以简单的让所有隐层统一数目，我们甚至还可以在模型过拟合的时候逐层增加神经元。   
  
一个简单的办法是构造一个层数和神经元数目都大于实际需要的模型，然后通过提前结束来防止过拟合或别的技术。这是“弹力裤”方法：与其浪费时间寻找，不如直接买一条大的，然后收缩到合适大小。
# 激活函数
大多数情况下隐层可以使用ReLU，或者它的变种。他比其他激活函数更加快速，但是GD算法就不好使用了。幸好的是，多数情况下都不会饱和。  
  
对于输出层，softmax是不错的选择，对于分类，而对于回归，几乎可以不适用激活函数。  
  
 